In [23]:
import math
import random
import time
from graph.graph import read_graph_from_csv

In [24]:
from stable_matching.TaskWorker import outward_satisfactory,overall_satisfactory,gamma_workers,individual_rationality_tasks,waste_pairwise,calculate_influence_workers
def estimate(Tasks, Workers, ise=False):
    if ise:
        for task_ in Tasks:
            task_.estimate()
    result_dict = {'fairness-pairwise': outward_satisfactory(Tasks, Workers, ise),
                   # 'overall_satisfactory': overall_satisfactory(tasks,workers,ise),
                   'individual_rationality': individual_rationality_tasks(Tasks),
                   'waste-pairwise': waste_pairwise(Tasks, Workers)}
    Sum = 0
    Max = 0
    Min = 1000
    for t_ in Tasks:
        if len(t_.students()) <=0:
            break
        q = calculate_influence_workers(t_.students(), t_.G, t_.values)
        Sum += q
        if q> Max:
            Max = q
        if q< Min:
            Min = q
    result_dict['average-density'] = Sum/len(Tasks)
    result_dict['maximum-density'] = Max
    result_dict['minimum-density'] = Min
    return result_dict

In [25]:
graph_file = '../graphs/dash/dash.csv'

In [26]:
m = 20 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [27]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [28]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task

mu = 1.0
sigma = 0.1

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 10
max_variance = math.ceil(avg_budget/2)

budgets = [random.randint(avg_budget-max_variance,avg_budget+max_variance) for _ in range(m)]
costs = {}
values ={}
Q = [0] * m
g = read_graph_from_csv(graph_file,0)
for i in range(m):
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = [random.uniform(0.1,2.0) for _ in range(n)]
    costs[i] = {}
    for j in range(n):
        costs[i][workers[j]] = X[j]
        
    # values of all workers
    values[i] = {}
    
    for v in g.nodes:
        values[i][v] = random.uniform(0.0, 1.0)
        Q[i] += values[i][v]
        

In [29]:
from tqdm import tqdm
tasks = []
graph_ids = random.sample(range(100),m)
with tqdm(total = m * 100, desc='generate tasks', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    for i in range(m):
        start = time.time()
        G = read_graph_from_csv(graph_file,graph_ids[i])
        budget = budgets[i]
        # generate hyper graph of reverse reachable set in graph G
        k = max_k(budget, costs[i])
        RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values[i], method= 'sq')
        pbar.set_postfix({'task': i,'time used':time.time()-start})
    
        #initialize tasks
        tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q[i]))
        tasks[i].initialize(costs[i])
        tasks[i].set_graph(G, values[i])
        pbar.update(100)

generate tasks: 100%|██████████████████| 2.00k/2.00k [00:57<00:00, 34.9B/s, task=19, time used=2.85]


In [30]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [36]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_max_cover()
generalized_da(tasks,workers)
estimate(tasks,workers)

{'fairness-pairwise': 0.7931034482758621,
 'individual_rationality': [[13, 6.270249658006151],
  [8, 1.6140006528385924],
  [9, 1.1812098773638422],
  [9, 4.505183829304954],
  [8, 4.91835019958624],
  [5, 4.115397608546226],
  [9, 3.2783781080906365],
  [5, 4.171247467749501],
  [12, 4.404985583590672],
  [7, 5.406867478630769],
  [8, 1.447144873999007],
  [12, 1.4788690786332936],
  [12, 6.627908997093858],
  [7, 1.7317306218758515],
  [14, 3.120816876431525],
  [7, 5.298986903988069],
  [13, 4.119402289034852],
  [7, 3.795329550475573],
  [11, 4.565721126699054],
  [8, 2.302578015967287]],
 'waste-pairwise': 4.827586206896552,
 'average-density': 8.000593834088182,
 'maximum-density': 15.7572423752883,
 'minimum-density': 3.5407480158995783}

In [31]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_budget()
generalized_da(tasks,workers)
estimate(tasks, workers)

{'fairness-pairwise': 1.0,
 'individual_rationality': [[13, 4.79461821634136],
  [8, 2.7618975212799532],
  [9, 6.54907476323231],
  [9, 6.8092967056818505],
  [8, 4.980108757694892],
  [5, 0],
  [9, 4.4154446706381005],
  [5, 4.22348582684037],
  [12, 3.9594209469632045],
  [7, 6.214367370622063],
  [8, 7.324920601799278],
  [12, 4.984319001544175],
  [12, 9.776597775069504],
  [7, 5.061223823564083],
  [14, 6.55098862514104],
  [7, 6.388083656216217],
  [13, 4.702760819412124],
  [7, 5.08252543053965],
  [11, 10.632717351485026],
  [8, 3.230853527793523]],
 'waste-pairwise': 0.0,
 'average-density': 1.744383440375206,
 'maximum-density': 16.593530633450467,
 'minimum-density': 1.9914517648986247}

In [ ]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_matroid(6)
generalized_da(tasks,workers)
estimate(tasks,workers)

In [ ]:
from stable_matching.stableMatching import heuristic
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
heuristic(tasks,workers,100)

In [35]:
flag = True
for task in tasks:
    for worker in task.students():
        if worker.task != task:
            flag = False
flag

True

In [34]:
estimate(tasks,workers)

{'fairness-pairwise': 1.0,
 'individual_rationality': [[13, 4.79461821634136],
  [8, 2.7618975212799532],
  [9, 6.549074763232309],
  [9, 6.80929670568185],
  [8, 4.980108757694893],
  [5, 0],
  [9, 4.4154446706381005],
  [5, 4.22348582684037],
  [12, 3.9594209469632045],
  [7, 6.214367370622064],
  [8, 7.324920601799278],
  [12, 4.984319001544175],
  [12, 9.776597775069504],
  [7, 5.061223823564083],
  [14, 6.55098862514104],
  [7, 6.388083656216218],
  [13, 4.702760819412124],
  [7, 5.08252543053965],
  [11, 10.632717351485026],
  [8, 3.230853527793523]],
 'waste-pairwise': 0.0,
 'average-density': 1.773869754407876,
 'maximum-density': 16.999112968469426,
 'minimum-density': 2.0429530447685558}